# Trabalho – Implementação de KNN - Método de Classificação para uma data set ( Iris ou Spam)


## Background

### KNN

O algoritmo K-Nearest Neighbors (KNN) é um método de aprendizado de máquina supervisionado utilizado para classificação e regressão. Ele opera com base na ideia de que itens semelhantes tendem a estar próximos uns dos outros. Para classificar um novo ponto de dados, o KNN calcula a distância entre esse ponto e os pontos de treinamento conhecidos e seleciona os K vizinhos mais próximos.

A classe ou valor alvo do novo ponto é determinado pela maioria das classes (no caso de classificação) ou pela média dos valores (no caso de regressão) dos K vizinhos mais próximos. O valor de K, que representa o número de vizinhos considerados, é um hiperparâmetro que pode ser ajustado para otimizar o desempenho do modelo.

A distância utilizada como métrica é a distância euclidiana, representada pela equação abaixo, onde $p$ e $p'$ são dois objetoros representados por vetores no espaço e n é a quantidade de dimensões.


$$ d(p,q) = \sqrt{\sum_{i=1}^{n}{(p - q)^2}} $$

### Data Set Iris

O conjunto de dados Iris é um dos conjuntos mais icônicos e amplamente utilizados em aprendizado de máquina e estatísticas. Ele contém informações sobre três espécies diferentes de plantas de íris: Setosa, Versicolor e Virginica. Para cada uma das 150 amostras de íris, são registradas quatro características: comprimento e largura das sépalas e pétalas. Esse conjunto de dados é frequentemente utilizado para tarefas de classificação e clustering, bem como para ilustrar conceitos fundamentais em análise de dados e visualização. Graças à sua simplicidade e clareza, o conjunto de dados Iris é uma escolha popular para iniciantes em aprendizado de máquina, bem como para pesquisadores que desejam explorar algoritmos e técnicas de análise de dados. O conjunto de dados pode ser acessado no repositório UCI Machine Learning e é uma valiosa ferramenta para estudos e experimentos na área de ciência de dados.

[Página do dataset](https://archive.ics.uci.edu/dataset/53/iris)

## Código

#### Importação das bibliotecas utilizadas

In [68]:
pip install scikit-learn

In [82]:
from sklearn.datasets import  load_iris
from sklearn.model_selection import train_test_split
from sklearn import metrics
import numpy as np
import math

### Algoritmo KNN


In [130]:
from sklearn.externals._packaging.version import Infinity
class KNN:
    def __init__(self,vizinhos: int):
        self.altera_quantidade_de_vizinhos(vizinhos)
        self.valores_de_treinamento = [[]]
        self.classes_de_treinamento = []
        self.dimensoes = 1
        self.total = 0

    def treinar(self, valores: np.ndarray[np.ndarray[int]], classes: np.ndarray[int]) -> None:
        self.total, self.dimensoes = valores.shape
        self.valores_de_treinamento = valores.copy()
        self.classes_de_treinamento = classes.copy()

    def altera_quantidade_de_vizinhos(self,vizinhos: int) -> None:
        self.quantidade_de_vizinhos = vizinhos

    def predizer(self, valores: np.ndarray[np.ndarray[int]]) -> list[int]:
        if len(self.valores_de_treinamento) == 0:
            raise Exception('É necessário treinar antes!')

        classes_classificadas = []
        for v in valores:
            classe = self.__calcular__(v)
            classes_classificadas.append(classe)

        return classes_classificadas


    def pontuar(self, valores: np.ndarray[np.ndarray[int]], classes: np.ndarray[int]) -> float:
        if len(valores) != len(classes):
            raise Exception('O tamanho dos valores está diferente do tamanho das classes!')

        preditos = self.predizer(valores)
        total = len(preditos)
        corretos = len([i for i in classes if classes[i] == preditos[i]])

        return corretos/total

    def __moda_com_desempate__(self,vizinhos: list[list[int]]) -> int:
        frequencia = {}
        menor_distancia = {}
        maior_frequencia = 0
        for i in range(0,len(vizinhos)):
            vizinho = vizinhos[i][0]
            distancia = vizinhos[i][1]
            classe = self.classes_de_treinamento[vizinho]
            if classe not in frequencia:
                menor_distancia[classe] = distancia
                frequencia[classe] = 1
            else:
                frequencia[classe] += 1
                if distancia < menor_distancia[classe]:
                    menor_distancia[classe] = distancia

            if frequencia[classe] > maior_frequencia:
                maior_frequencia = frequencia[classe]

        print(frequencia)
        print(menor_distancia)
        print(maior_frequencia)

        moda = [classe for classe in frequencia if frequencia[classe] == maior_frequencia]

        if len(moda) == 1:
            return moda[0]
        else:
            menor_distancia_da_moda = Infinity
            menor_classe = None
            for classe in moda:
                if menor_distancia[classe] < menor_distancia_da_moda:
                    menor_distancia_da_moda = menor_distancia[classe]
                    menor_classe = classe


    def __calcular__(self,valor: np.ndarray[int]) -> int:
        vizinhos = []
        precisa_ordenar = True
        for i in range(0, self.total):
            treinado = self.valores_de_treinamento[i]
            somatorio = 0
            for dimensao in range(0,self.dimensoes):
                somatorio += pow(valor[dimensao] - treinado[dimensao],2)
            distancia = math.sqrt(somatorio)

            if len(vizinhos) < self.quantidade_de_vizinhos:
                vizinhos.append([i,distancia])
            else:
                if precisa_ordenar:
                    vizinhos.sort(key=lambda i: i[1])
                    precisa_ordenar = False
                mais_distante = vizinhos[-1][1]
                if distancia < mais_distante:
                    vizinhos.pop()
                    vizinhos.append([i,distancia])
                    precisa_ordenar = True

        print(vizinhos)
        classe = self.__moda_com_desempate__(vizinhos)
        return classe





### Íris

#### Carregamento do dataset

In [50]:
VALORES, CLASSES = load_iris(return_X_y=True)

#### Divide o dataset em treinamento e teste

In [39]:
VALORES_DE_TREINAMENTO, VALORES_DE_TESTE, CLASSES_DE_TREINAMENTO, CLASSES_DE_TESTE = train_test_split(VALORES, CLASSES, test_size=0.7, random_state=3)

#### Número de vizinhos

In [103]:
VIZINHOS = 5

#### Roda KNN com Iris

In [135]:
knn = KNN(1)
knn.treinar(VALORES_DE_TREINAMENTO, CLASSES_DE_TREINAMENTO)
teste = 10
print(f'teste: {CLASSES_DE_TESTE[:teste]}')
knn.pontuar(VALORES_DE_TESTE[:teste],CLASSES_DE_TESTE[:teste])

teste: [0 0 0 0 0 2 1 0 2 1]
[[33, 0.14142135623730978]]
{0: 1}
{0: 0.14142135623730978}
1
[[33, 0.24494897427831802]]
{0: 1}
{0: 0.24494897427831802}
1
[[15, 0.282842712474619]]
{0: 1}
{0: 0.282842712474619}
1
[[13, 0.3605551275463989]]
{0: 1}
{0: 0.3605551275463989}
1
[[20, 0.3605551275463992]]
{0: 1}
{0: 0.3605551275463992}
1
[[6, 0.5477225575051664]]
{2: 1}
{2: 0.5477225575051664}
1
[[29, 0.24494897427831766]]
{1: 1}
{1: 0.24494897427831766}
1
[[33, 0.264575131106459]]
{0: 1}
{0: 0.264575131106459}
1
[[8, 0.49999999999999967]]
{2: 1}
{2: 0.49999999999999967}
1
[[9, 0.42426406871192845]]
{1: 1}
{1: 0.42426406871192845}
1


1.0